In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score,roc_auc_score,roc_curve, auc,classification_report
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf

In [24]:
data = pd.read_csv('patient_info.csv')

# **PRE PROCESSING**

In [5]:
col_names = ['Age','Gender','TB_total_bilirubin', 'DB_Direct_Bilirubin','Alkphos_Alkaline_Phosphotase', 'Sgpt_Alamine_Aminotransferase',
       'Sgot_Aspartate_Aminotransferase', 'TP_Total_Protiens', 'ALB_Albumin', 'A/G_Ratio','Selector Field']
len(col_names)

11

In [6]:
data.columns = col_names

In [7]:
data.isnull().sum()

Age                                0
Gender                             0
TB_total_bilirubin                 0
DB_Direct_Bilirubin                0
Alkphos_Alkaline_Phosphotase       0
Sgpt_Alamine_Aminotransferase      0
Sgot_Aspartate_Aminotransferase    0
TP_Total_Protiens                  0
ALB_Albumin                        0
A/G_Ratio                          4
Selector Field                     0
dtype: int64

In [9]:
data['A/G_Ratio'].fillna(round(data['A/G_Ratio'].mean(),2),inplace=True)

In [10]:
data.isnull().sum()

Age                                0
Gender                             0
TB_total_bilirubin                 0
DB_Direct_Bilirubin                0
Alkphos_Alkaline_Phosphotase       0
Sgpt_Alamine_Aminotransferase      0
Sgot_Aspartate_Aminotransferase    0
TP_Total_Protiens                  0
ALB_Albumin                        0
A/G_Ratio                          0
Selector Field                     0
dtype: int64

In [11]:
data['Selector Field'] = data['Selector Field'].map(lambda x:0 if x==2 else 1)  # 0-no disease, 1-disease
data['Gender'] = data['Gender'].map(lambda x:0 if x=="Male" else 1)
value_table = pd.crosstab(data['Gender'],data['Selector Field'])
value_table.columns = ["No Disease","Disease Present"]
value_table.index = ["Male","Female"]
print(value_table)

        No Disease  Disease Present
Male           117              324
Female          50               92


In [12]:
X = data.drop('Selector Field',axis=1)
y = data['Selector Field']

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X)

# **TRAINING**

In [14]:
Xtrain, Xtest, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

print(len(X_train))
print(len(y_train))

408
408


In [15]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.5 MB/s eta 0:00:00


# **MODEL SELECTION**

In [18]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

num_features_to_select = X.shape[1]

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.choice, [True, False])
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=num_features_to_select)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
def evaluate(individual):
    selected_features = [feature for feature, select in zip(X_train.T, individual) if select]
    if not any(individual) or len(selected_features) == 0:
        return 0.0,
    clf = RandomForestClassifier(random_state=0)
    clf.fit(X_train[:, individual], y_train)
    y_pred = clf.predict(X_test[:, individual])
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=58)

generations = 15
for gen in range(generations):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.3)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

best_individual = tools.selBest(population, k=1)[0]
selected_features_indices = [i for i, select in enumerate(best_individual) if select]
print("Best Individual (Feature Selection):", selected_features_indices)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Best Individual (Feature Selection): [2, 3, 4, 5, 6]


# **ANN**

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

classifier = Sequential()
classifier.add(Dense(units=128, activation='relu', input_dim=len(selected_features_indices)))
classifier.add(Dropout(rate=0.6))
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(rate=0.6))
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dropout(rate=0.6))
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(rate=0.4))
classifier.add(Dense(units=1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               768       
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               3

In [20]:
X_new = X_train[:, selected_features_indices]
selected_columns = data.columns[selected_features_indices]
X_new = pd.DataFrame(X_new, columns=selected_columns)
print(X_new.head())

   TB_total_bilirubin  DB_Direct_Bilirubin  Alkphos_Alkaline_Phosphotase  \
0           -0.379496            -0.448134                     -0.382407   
1           -0.395354            -0.448134                     -0.430120   
2           -0.347781            -0.410536                     -0.492147   
3            3.172624             4.214006                      1.411608   
4           -0.379496            -0.448134                     -0.659143   

   Sgpt_Alamine_Aminotransferase  Sgot_Aspartate_Aminotransferase  
0                      -0.336702                        -0.454193  
1                      -0.421733                        -0.515861  
2                      -0.287101                        -0.380192  
3                      -0.194984                        -0.287691  
4                      -0.329616                        -0.454193  


In [21]:
X_newtest = X_test[:, selected_features_indices]
selected_columns = data.columns[selected_features_indices]
X_newtest = pd.DataFrame(X_newtest, columns=selected_columns)
print(X_newtest.head())

   TB_total_bilirubin  DB_Direct_Bilirubin  Alkphos_Alkaline_Phosphotase  \
0           -0.236777            -0.260144                      2.771433   
1           -0.379496            -0.448134                      0.113810   
2           -0.284350            -0.260144                      0.915391   
3            0.001088             0.078237                     -0.100899   
4            1.793006             1.882936                      0.381004   

   Sgpt_Alamine_Aminotransferase  Sgot_Aspartate_Aminotransferase  
0                       0.130968                        -0.281524  
1                      -0.166640                        -0.164356  
2                      -0.060352                        -0.034854  
3                      -0.265843                         0.137815  
4                      -0.145383                        -0.115022  


In [22]:
history = classifier.fit(X_new, y_train, epochs=50, shuffle=False, validation_split=0.1)

Epoch 1/50
12/12 [==============================] - 1s 19ms/step - loss: 0.6041 - accuracy: 0.7084 - val_loss: 0.6160 - val_accuracy: 0.6341
Epoch 2/50
12/12 [==============================] - 0s 6ms/step - loss: 0.5758 - accuracy: 0.7221 - val_loss: 0.5884 - val_accuracy: 0.6341
Epoch 3/50
12/12 [==============================] - 0s 6ms/step - loss: 0.5728 - accuracy: 0.7221 - val_loss: 0.5697 - val_accuracy: 0.6341
Epoch 4/50
12/12 [==============================] - 0s 4ms/step - loss: 0.5550 - accuracy: 0.7193 - val_loss: 0.5591 - val_accuracy: 0.6341
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 0.5339 - accuracy: 0.7221 - val_loss: 0.5519 - val_accuracy: 0.6341
Epoch 6/50
12/12 [==============================] - 0s 4ms/step - loss: 0.5380 - accuracy: 0.7221 - val_loss: 0.5464 - val_accuracy: 0.6341
Epoch 7/50
12/12 [==============================] - 0s 4ms/step - loss: 0.5536 - accuracy: 0.7221 - val_loss: 0.5441 - val_accuracy: 0.6341
Epoch 8/50
12/12 [=

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_newtest)
y_pred = y_pred > 0.5

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred)*100))

6/6 [==============================] - 0s 3ms/step
Confusion matrix:
[[  0  50]
 [  1 124]]
Accuracy: 70.86%
